# Template 8 - Genetic Algorithm

Use this notebook as a template for Part 5 Optimization Algorithm Analysis of Coursework 2.

<span style = "color:limegreen;"> Before beginning I would suggest making a copy of this file to avoid any Git conflicts!</span>

## Load all the necessary Python packages
All packages should work with Conda environment if installed on your machine. Otherwise all necessary packages can be installed in a virtual environment (.venv) in VS Code using: Ctrl+Shift+P > Python: Create Environment > Venv > Python 3.12.x > requirements.txt

In [ ]:
import json
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
from pathlib import Path
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.selection.rnd import RandomSelection
from pymoo.termination import get_termination
from pymoo.optimize import minimize
import time

from src.runEnergyPlus import run_energyPlus
from src.saveResults import saveNSGA2History, saveNSGA2Optimal

## 1. Getting Started
### 1.1 Enter the general parameters for this run.

<span style="color:lime;"> These should be the same from previous exercises. Ensure they are correct.</span>

<span style="color:lime;">Give a unique *saveName for each run*</span>

In [ ]:
# Enter a save name for this run
saveName = ???

# Enter the path to the directory with your EnergyPlus executable. Enter the full path separated by commas.
ep_dir = Path("c:\\", "EnergyPlusV25-1-0")

# The weather file to be used for this batch of simulations. This file should be located in the src/weatherData/ directory.
weatherFile = "GBR_ENG_London.Wea.Ctr-St.James.Park.037700_TMYx.2004-2018.epw"

# The baseline file to be used for this simulation. This file should be located in the idfs/ directory
idf_file = "1-storey_baseline.idf"

# The parameters file to be used as part of this simulation
parameters_file = ???


Create the full paths for the idf and weather files and confirm they both exist. Else an exception will be created.

In [ ]:
baseline_idf_path = Path("idfs", idf_file)
weather_file_path = Path("weatherData", weatherFile)

if not ep_dir.exists():
    raise Exception (f"Could not find energyPlus executable at {ep_dir}.")
if not baseline_idf_path.exists():
    raise Exception (f"Could not find idf_file at {baseline_idf_path}.")
if not weather_file_path.exists():
    raise Exception (f"Could not find weather_file at {weather_file_path}.")

print (f"The EnergyPlus directory is: {ep_dir}.")
print (f"The baseline idf file is: {baseline_idf_path}.")
print (f"The weather file is: {weather_file_path}.")

## 1.2 Load the parameters file

In [ ]:
parameters_file_path = Path("simulationParameters", parameters_file)

if not parameters_file_path.exists():
    raise Exception (f"Could not find the parameters_file at {parameters_file_path}.")

with open (parameters_file_path) as f:
    parameters = json.load(f)

print (f"Name                      TYPE        VALUES")
for k,v in parameters.items():
    print (f"{k:<26}{v['type']:<12}{v['values']}")

## 2. Setting Up Pymoo

### 2.1 Problem Class

Remember to add the correct objective functions at the bottom of this step

In [ ]:
class Problem_EnergyPlus(Problem): # Note that this needs to be a Problem and not an ElementwiseProblem to run in parallel
    # Initialise the class with these default arguments
    def __init__(self, 
                 ep_dir = None,
                 idf_path = None, 
                 weather_file_path = None,
                 parameters = None,
                 n_obj = 2,
                 n_ieq_constr = 0, 
        ):

        #Inherit attributes and methods from the parent class Problem using super()
        super().__init__()

        # Add the number of objects and inequality constraints as attributes        
        self.n_obj = n_obj
        self.n_ieq_constr = n_ieq_constr

        # Store the information about the EnergyPlus parameters as attributes
        self.parameterNames = list(parameters.keys())
        self.n_var = len(parameters)
        self.xl = np.array([min(x["values"]) for x in parameters.values()])
        self.xu = np.array([max(x["values"]) for x in parameters.values()])


    def _evaluate(self, x, out, *args, **kwargs):
        # prepare the parameters for the multiprocessing pool
        inputs = pd.DataFrame(x, columns = self.parameterNames)
        inputs = inputs.to_dict("records")
        inputs = [(ep_dir, baseline_idf_path, weather_file_path, inputs[i], i) for i in range(len(x))]

        # Determine the number of processors to use
        n_processors = os.cpu_count()

        t0 = time.time()
        with Pool(processes = n_processors) as pool:
            returnValues = pool.starmap(run_energyPlus, inputs)
        t1 = time.time()

        # Un pack the results from the batch simulation
        returnCodes = [i[0] for i in returnValues]
        hourlyResults = [i[1] for i in returnValues]
        resilienceResults = [i[2] for i in returnValues]

        # Check if any simulations had errors
        errors = [x.args for x in returnCodes if x.returncode == 1]
        if len(errors) > 0:
            print (f"The following {len(errors)} simulations had errors:")
            for error in errors:
                print (f"\t{error}")
        else:
            print (f"All simulations completed successfully in {t1 - t0:.4f} s.")

        # Putting both the results dictionaries into a dataframe and concatenating them together
        df = pd.DataFrame(hourlyResults)
        df2 = pd.DataFrame(resilienceResults)

        df = pd.concat([df, df2], axis = 1)

        # Return the results of each objective to the output dictionary 
        out["F"] = ???


Create an instance of the problem using the default arguments for the file paths and the parameters to be used.

In [ ]:
problem = Problem_EnergyPlus(
    ep_dir = ep_dir,
    idf_path = baseline_idf_path,
    weather_file_path = weather_file_path, 
    parameters = parameters,
    )

## 2.2 Setting up the Algorithm
In this step, determine the number of individuals and offsprings

In [ ]:
algorithm = NSGA2(
    pop_size = ???,
    n_offsprings = ???,
    sampling = FloatRandomSampling(),
    selection = RandomSelection(),
    crossover = SBX(),
    mutation = PM(eta=20),
    eliminate_duplicates = True
)

### 2.3 Setting up the termination criteria. 
Select an appropriate value from one of these options

In [ ]:
#termination = get_termination("n_gen", 30)
#termination = get_termination("n_eval", 500)
#termination = get_termination("time", "00:00:05")

### 2.4 Running the algorithm as a minimization program

In [ ]:
if __name__ == "__main__":
    res = minimize(problem,
                algorithm,
                termination,
                seed = 314,
                save_history = True,
                verbose = True,
                )


Access the results and convert to a dataframe
Remember to add the correct objective functions below.

In [ ]:
# Access the arrays for X and F the same way you would for a class attribute
X = res.X
F = res.F

# Convert each into a dataframe
X = pd.DataFrame(X, columns = parameters.keys())
F = pd.DataFrame(F, columns = ["???", "???"])

### 2.6 Save the Results
As a final step, let's save the results so we can analyze them another time without having to re-run the simulations.

In [ ]:
# Save the history of X and F
history_X, history_F = saveNSGA2History(res.history, parameters, saveName)

# Save the final optimal set of X and F
final_generation =  history_F.generation.max()
X, F = saveNSGA2Optimal(X, F, final_generation, saveName)

### 3. Analysis

Perform your analysis here or in a separate notebook.